In [ ]:
# Third-party
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from gala.dynamics import mockstream as ms
from pyia import GaiaData

For plotting data:

In [ ]:
tbl = Table.read('../../gd1-dr2/output/rrl_bhb_bs_rgb_master.fits')

g = GaiaData(tbl)

rv = tbl['rv'].copy()
rv[~np.isfinite(rv)] = 0.

c = coord.SkyCoord(ra=tbl['ra']*u.deg, 
                   dec=tbl['dec']*u.deg,
                   distance=coord.Distance(distmod=tbl['DM']),
                   pm_ra_cosdec=tbl['pmra']*u.mas/u.yr,
                   pm_dec=tbl['pmdec']*u.mas/u.yr,
                   radial_velocity=rv*u.km/u.s)
c_gd1 = c.transform_to(gc.GD1)

# Only take stars with phi1 > -80
phi1_mask = c_gd1.phi1.wrap_at(180*u.deg) > -80*u.deg
c_gd1 = c_gd1[phi1_mask]
c = c[phi1_mask]
g = g[phi1_mask]
rv = rv[phi1_mask]
tbl = tbl[phi1_mask]

dist = coord.Distance(distmod=np.random.normal(tbl['DM'], tbl['DM_error'], 
                                               size=(10000, len(tbl))))

In [ ]:
data = Table()
data['phi1'] = c_gd1.phi1.wrap_at(180*u.deg)
data['phi2'] = c_gd1.phi2
data['distance'] = np.nanmean(dist, axis=0).to(u.kpc)
data['pm_phi1_cosphi2'] = c_gd1.pm_phi1_cosphi2
data['pm_phi2'] = c_gd1.pm_phi2
data['radial_velocity'] = tbl['rv'] * u.km/u.s
data = data.filled(fill_value=0)

---

In [ ]:
galcen_frame = coord.Galactocentric(galcen_distance=8.1*u.kpc)

In [ ]:
# mw = gp.MilkyWayPotential()
# H = gp.Hamiltonian(mw)

pot = gp.CCompositePotential()
pot['disk'] = gp.MiyamotoNagaiPotential(**{'a': 3.0, 'b': 0.28, 'm': 128682847895.60304}, units=galactic)
pot['halo'] = gp.NFWPotential(**{'a': 1.0, 'b': 1, 'c': 1.45342154251265,
                                 'm': 601303614092.3599, 'r_s': 16.013386267833617},
                              units=galactic)
H = gp.Hamiltonian(pot)

In [ ]:
prog_w0 = gd.PhaseSpacePosition([15., 0, 0]*u.kpc,
                                [0, 180, 0.]*u.km/u.s)
prog_mass = 1e4 * u.Msun
prog_pot = gp.PlummerPotential(prog_mass, b=2*u.pc, units=galactic)

In [ ]:
c = gc.GD1Koposov10(**{'phi2': 0.12716889157882055*u.deg,
                       'distance': 8.81723527532562*u.kpc,
                       'pm_phi1_cosphi2': -11.576122612503559*u.mas/u.yr,
                       'pm_phi2': -2.5105753010704452*u.mas/u.yr,
                       'radial_velocity': -162.1453863232979*u.km/u.s},
                    phi1=-20*u.deg)
w0 = gd.PhaseSpacePosition(c.transform_to(galcen_frame).cartesian)

In [ ]:
# gen = ms.MockStreamGenerator(df, H)

In [ ]:
def make_five_panel(stream_c):
    fig, axes = plt.subplots(5, 1, figsize=(8, 12), 
                             sharex=True)

    for i, name in enumerate(['phi2', 'distance', 'pm_phi1_cosphi2', 'pm_phi2', 'radial_velocity']):
        ax = axes[i]

        ax.plot(data['phi1'], data[name], 
                marker='o', ls='none', color='tab:blue', ms=2)

        ax.plot(stream_c.phi1.wrap_at(180*u.deg).degree,
                getattr(stream_c, name).value, 
                marker='o', ls='none', color='k', ms=2.5, alpha=0.3, zorder=-100)
        ax.set_ylabel(name, fontsize=12)

    ax.set_xlim(-100, 20)
    axes[0].set_ylim(-10, 5)
    axes[1].set_ylim(5, 15)

    fig.set_facecolor('w')
    return fig

In [ ]:
for dist in np.arange(24., 26+1e-3, 0.1):
    for sag_m in np.exp(np.linspace(np.log(2), np.log(60)+1e-3, 9)) * 1e9:
        sag_c = coord.SkyCoord(ra=283.8313*u.deg, dec=-30.5453*u.deg,
                               distance=dist*u.kpc, 
                               pm_ra_cosdec=-2.692*u.mas/u.yr,
                               pm_dec=-1.359*u.mas/u.yr,
                               radial_velocity=142.6*u.km/u.s)
        sag_w0 = gd.PhaseSpacePosition(sag_c.transform_to(galcen_frame).data)
        nbody = gd.DirectNBody(sag_w0, 
                               particle_potentials=[gp.HernquistPotential(m=sag_m, c=2.5*u.kpc, units=galactic)], 
                               external_potential=H.potential, frame=H.frame)
        
        df = ms.FardalStreamDF(random_state=np.random.RandomState(42))
        gen = ms.MockStreamGenerator(df, H, progenitor_potential=prog_pot)
        stream, _ = gen.run(w0, prog_mass, dt=-1, n_steps=4000, 
                            nbody=nbody, release_every=4)

        # fig = stream.plot(color='k', alpha=0.1, lw=0)
        # fig.set_facecolor('w')
        # for ax in fig.axes:
        #     ax.set_xlim(-25, 25)
        #     ax.set_ylim(-25, 25)
        # fig.savefig('../plots/xyz_dist{:.1f}_sag{:.1f}e9.png'.format(dist, sag_m/1e9), dpi=200)
        # plt.close(fig)
    
        stream_c = stream.to_coord_frame(gc.GD1Koposov10, galcen_frame)
        fig = make_five_panel(stream_c)
        fig.savefig('../plots/obs_dist{:.1f}_sag{:.1f}e9.png'.format(dist, sag_m/1e9), dpi=200)
        plt.close(fig)